In [1]:
# ---- Libraries -----

from fenics import *
import numpy as np
import matplotlib.pyplot as plt
import math
from dolfin import *
!pip install scipy
import scipy

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [2]:
# ---- Parameters-----

# File name
filename = "test1_3D"       # [[0.01, 0, 0], [0, 0.01, 0], [0, 0, 0.01]], p=1
#filename = "test2_3D"      # [[0.01, 0, 0], [0, 0.01, 0], [0, 0, 0.01]], p=2
#filename = "test3_3D"      # [[0.01, 0.01, 0], [0.01, 0.01, 0], [0, 0, 0.01]], p=1
#filename = "test4_3D"      # [[0.01, 0.01, 0], [0.01, 0.01, 0], [0, 0, 0.01]], p=2

# Diffusion (choose the coefficients accordingly to the test)
d11 = 0.01
d12 = 0
d13 = 0

d21 = 0
d22 = 0.01
d23 = 0

d31 = 0
d32 = 0
d33 = 0.01

components_D = [[d11, d12, d13], [d21, d22, d23], [d31, d32, d33]]


# Time
t0 = 0
dt = 1e-5
num_steps = 10
T = num_steps*dt


# Functional space
p = 1               # polynomial degree 

# Stabilization
alpha = 3

# Initial datum
u0 = Expression('(1-cos(2*pi*x[0]))*(1-cos(2*pi*x[1]))*(1-cos(2*pi*x[2]))', degree = p) # bump

# Exact solution
u_ex = Expression('exp(-t)*(1-cos(2*pi*x[0]))*(1-cos(2*pi*x[1]))*(1-cos(2*pi*x[2]))', t = 0, degree = p)

# Force
f = Expression('exp(-t)*(cos(2*pi*x[0]) - 1)*(cos(2*pi*x[1]) - 1)*(cos(2*pi*x[2]) - 1) + exp(-t)*(exp(-t)*(cos(2*pi*x[0]) - 1)*(cos(2*pi*x[1]) - 1)*(cos(2*pi*x[2]) - 1) + 1)*(cos(2*pi*x[0]) - 1)*(cos(2*pi*x[1]) - 1)*(cos(2*pi*x[2]) - 1) - 4*d11*pi*pi*exp(-t)*cos(2*pi*x[0])*(cos(2*pi*x[1]) - 1)*(cos(2*pi*x[2]) - 1) - 4*d22*pi*pi*exp(-t)*cos(2*pi*x[1])*(cos(2*pi*x[0]) - 1)*(cos(2*pi*x[2]) - 1) - 4*d33*pi*pi*exp(-t)*cos(2*pi*x[2])*(cos(2*pi*x[0]) - 1)*(cos(2*pi*x[1]) - 1) + 4*d12*pi*pi*exp(-t)*sin(2*pi*x[0])*sin(2*pi*x[1])*(cos(2*pi*x[2]) - 1) + 4*d13*pi*pi*exp(-t)*sin(2*pi*x[0])*sin(2*pi*x[2])*(cos(2*pi*x[1]) - 1) + 4*d21*pi*pi*exp(-t)*sin(2*pi*x[0])*sin(2*pi*x[1])*(cos(2*pi*x[2]) - 1) + 4*d23*pi*pi*exp(-t)*sin(2*pi*x[1])*sin(2*pi*x[2])*(cos(2*pi*x[0]) - 1) + 4*d31*pi*pi*exp(-t)*sin(2*pi*x[0])*sin(2*pi*x[2])*(cos(2*pi*x[1]) - 1) + 4*d32*pi*pi*exp(-t)*sin(2*pi*x[1])*sin(2*pi*x[2])*(cos(2*pi*x[0]) - 1)', t = 0, d11 = d11, d12 = d12, d13 = d13, d21 = d21, d22 = d22, d23 = d23, d31 = d31, d32 = d32, d33 = d33, degree = p)


# Neumann boundary condition
gN = Expression('0*x[0] + 0*x[1] + 0*x[2]', degree = p)

In [3]:
# ---- Computations ----

# Diffusion tensor
D = as_matrix(components_D)
norm_D = np.linalg.norm(np.matrix(components_D))

In [4]:
# ---- Convergence test initialization ----

N = np.power(2,[2,3,4,5,6,7])
err_H1 = np.zeros(len(N))
err_L2 = np.zeros(len(N))

In [5]:

for i in range(1,len(N)):

    # ---- Geometry ----

    nx = ny = nz = N[i]

    # Create mesh and define function space
    hx = 1/nx
    hy = 1/ny
    hz = 1/nz
    h = sqrt(hx*hx + hy*hy + hz*hz)     # square's diagonal (diam(elem))
    mesh = UnitCubeMesh(nx, ny, nz)
    V = FunctionSpace(mesh, "DG", p)

    # DG stabilization coefficient
    sigma = norm_D + p*p/h*alpha

    # Projection of u_0 on V
    u0h = interpolate(u0, V)

    # ---- Variational problem ----

    # Solution
    u_n = u0h
    u = TrialFunction(V)

    # Test function
    v = TestFunction(V)

    # Outer normal
    n = FacetNormal(mesh)

    # DG bilinear form
    B = inner(D*grad(u), grad(v))*dx - inner(avg(D*grad(u)), jump(v, n))*dS -\
        inner(avg(D*grad(v)), jump(u, n))*dS +\
        sigma*inner(jump(u, n), jump(v, n))*dS

    # Final problem
    a = u*v*dx + dt*B - dt*(1-u_n)*u*v*dx 
    F = u_n*v*dx + gN*v*ds + f*v*dx

    # ---- Time-stepping ----

    u = Function(V)

    # Initial time
    t = t0
    
    # Create VTK file for saving solution
    vtkfile = File('solutions_convergence/' + filename + '/' + filename +'.pvd')

    for n in range(num_steps):
        
        # Update current time
        t += dt

        # Compute solution
        solve(a == F, u)

        # Update previous solution
        u_n.assign(u)
        vtkfile << (u_n, t)

    err_H1[i] = errornorm(u_ex, u, 'H1')
    err_L2[i] = errornorm(u_ex, u, 'L2')
    

Calling FFC just-in-time (JIT) compiler, this may take some time.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
*** Warning: Degree of exact solution may be inadequate for accurate result in errornorm.
*** Warning: Degree of exact solution may be inadequate for accurate result in errornorm.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
*** Warning: Degree of exact solut

RuntimeError: 

*** -------------------------------------------------------------------------
*** DOLFIN encountered an error. If you are not able to resolve this issue
*** using the information listed below, you can ask for help at
***
***     fenics-support@googlegroups.com
***
*** Remember to include the error message listed below and, if possible,
*** include a *minimal* running example to reproduce the error.
***
*** -------------------------------------------------------------------------
*** Error:   Unable to successfully call PETSc function 'KSPSolve'.
*** Reason:  PETSc error code is: 76 (Error in external library).
*** Where:   This error was encountered inside /build/dolfin-P9xfrX/dolfin-2019.2.0~git20220407.d29e24d/dolfin/la/PETScKrylovSolver.cpp.
*** Process: 0
*** 
*** DOLFIN version: 2019.2.0.dev0
*** Git changeset:  unknown
*** -------------------------------------------------------------------------


In [8]:
# Compute the error rates
h = np.sqrt(1/N*1/N + 1/N*1/N + 1/N*1/N) 
order_H1 = np.log(err_H1[1:(len(err_H1)-1)]/err_H1[2:len(err_H1)]) / np.log(h[1:(len(h)-1)]/h[2:len(h)])
order_L2 = np.log(err_L2[1:(len(err_L2)-1)]/err_L2[2:len(err_L2)]) / np.log(h[1:(len(h)-1)]/h[2:len(h)])

array([0.4330127 , 0.21650635, 0.10825318, 0.05412659, 0.02706329,
       0.01353165])